<a href="https://colab.research.google.com/github/Alex-2911/Basketball-Projection/blob/main/_3_01122024_lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
ROLLING_WINDOW_SIZE = 7
current_season = 2025

In [34]:
# import os
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define base directory in Google Drive
#DATA_DIR = os.path.join("/content/drive/My Drive", "NBA_Script", "2025", "Gathering_Data", "data")
#SRC_DIR = os.path.join("/content/drive/My Drive", "NBA_Script", "2025", "Gathering_Data", "Whole_Statistic")

# Define directories
season_dir ="/content/drive/My Drive/NBA_Script/2025"
base_dir = "/content/drive/My Drive/NBA_Script/2025/Gathering_Data"
STANDINGS_DIR = os.path.join(base_dir, f"{current_season}_standings")
SCORES_DIR = os.path.join(base_dir, f"{current_season}_scores")
SRC_DIR = os.path.join(base_dir, f"Whole_Statistic")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd
import datetime
import numpy as np
import lightgbm as lgb
import os


import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [7]:
import os
import pandas as pd
import glob
import datetime

# Define directory and date format
#directory = "D:\\1. Python\\1. NBA Script\\2025\\Gathering_Data\\Next_Game\\"
directory = "/content/drive/My Drive/NBA_Script/2025/Gathering_Data/Next_Game"

today = datetime.datetime.now().strftime("%Y-%m-%d")

# Check if file exists
# Added '/' between directory and filename to construct the correct path
file_path = os.path.join(directory, f"games_df_{today}.csv")
if not os.path.exists(file_path):
    # List files and pick the latest one
    files = sorted(glob.glob(os.path.join(directory, f"games_df_*.csv")))  # Updated here as well
    if files:
        file_path = files[-1]  # Use the latest available file
        print(f"Using the latest file: {file_path}")
    else:
        print("No files found in the directory.")
        exit()

# Proceed to read the file
games_df = pd.read_csv(file_path, index_col=0)
print(games_df.head(60).to_string(index=False))

home_team away_team  game_date
      BRK       ORL 2024-12-01
      MEM       IND 2024-12-01
      CLE       BOS 2024-12-01
      NYK       NOP 2024-12-01
      TOR       MIA 2024-12-01
      HOU       OKC 2024-12-01
      UTA       LAL 2024-12-01
      POR       DAL 2024-12-01
      SAC       SAS 2024-12-01
      LAC       DEN 2024-12-01


In [8]:
import os
import pandas as pd
import datetime
from sklearn.preprocessing import MinMaxScaler

# Constants
#DATA_DIRECTORY = "D:\\1. Python\\1. NBA Script\\2025\\Gathering_Data\\Whole_Statistic\\"
#SRC_DIR = os.path.join(base_dir, f"Whole_Statistic")
TODAY = (datetime.datetime.now() - datetime.timedelta(days=0)).strftime("%Y-%m-%d")

df_path = os.path.join(SRC_DIR, f"nba_games_{TODAY}.csv")

# Function to find the most recent file in the directory if the desired one is not available
def get_latest_available_file(data_directory, prefix="nba_games_", extension=".csv"):
    """Returns the latest available CSV file matching the pattern."""
    available_files = [f for f in os.listdir(data_directory) if f.startswith(prefix) and f.endswith(extension)]
    if available_files:
        latest_file = max(available_files, key=lambda f: os.path.getctime(os.path.join(data_directory, f)))
        return os.path.join(data_directory, latest_file)
    return None

# Check if the specific file for today exists; if not, fallback to the most recent available file
if not os.path.exists(df_path):
    print(f"File for {TODAY} not found. Searching for the latest available file...")
    df_path = get_latest_available_file(DATA_DIRECTORY)
    if df_path:
        print(f"Using the latest available file: {df_path}")
    else:
        raise FileNotFoundError(f"No suitable file found in the directory: {DATA_DIRECTORY}")

# Proceed with loading the data
df = pd.read_csv(df_path, index_col=0)
print(df)#.tail())  # Display a portion of the data

# Function to add a target column
def add_target(group):
    """Adds a target column to the DataFrame group based on the 'won' column."""
    group['target'] = group['won'].shift(-1)
    return group

def preprocess_nba_data():
    # Load the data
    df = pd.read_csv(df_path, index_col=0)

    # Sort by date
    df = df.sort_values("date")

    # Apply the preprocessing function to each team group
    df = df.groupby('team').apply(add_target)

    # Handle missing values
    df['target'].fillna(2, inplace=True)
    df['target'] = df['target'].astype(int)

    # Identify and remove columns with null values
    nulls = pd.isnull(df).sum()
    nulls = nulls[nulls > 0]
    valid_columns = df.columns[~df.columns.isin(nulls.index)]
    df = df[valid_columns].copy()

    return df

if __name__ == "__main__":
    df = preprocess_nba_data()

    # Columns to be excluded from scaling
    removed_columns = ["season", "date", "won", "target", "team", "team_opp"]

    # Selecting columns that are not in the 'removed_columns' list
    selected_columns = df.columns[~df.columns.isin(removed_columns)]

    # Initialize the MinMaxScaler
    scaler = MinMaxScaler()

    # Scale the selected columns and update the DataFrame
    df[selected_columns] = scaler.fit_transform(df[selected_columns])

    #df.to_csv("D:\\1. Python\\1. NBA Script\\2025\\Gathering_Data\\Whole_Statistic\\df_orig.csv", index=False)


    # Optionally, save the scaled DataFrame
    # output_path = os.path.join(DATA_DIRECTORY, f"scaled_nba_games_{TODAY}.csv")
    # df.to_csv(output_path, index=False)
    # print(f"Scaled data saved to {output_path}")


         fg   fga      fg%    3p   3pa      3p%    ft   fta      ft%   orb  \
mp.1                                                                         
240.0  37.0  96.0  385.000  12.0  29.0  414.000  20.0  26.0  769.000  23.0   
240.0  37.0  82.0  451.000   8.0  27.0  296.000  12.0  15.0    0.800   7.0   
240.0  38.0  94.0  404.000   9.0  29.0    0.310  10.0  17.0  588.000  11.0   
240.0  37.0  87.0  425.000   7.0  19.0  368.000  16.0  23.0  696.000   7.0   
240.0  35.0  83.0  422.000   6.0  18.0  333.000  19.0  27.0  704.000   8.0   
...     ...   ...      ...   ...   ...      ...   ...   ...      ...   ...   
NaN    44.0  86.0    0.512  16.0  35.0    0.457  20.0  25.0    0.800   7.0   
NaN    38.0  92.0    0.413  14.0  40.0    0.350  15.0  18.0    0.833   9.0   
NaN    38.0  79.0    0.481  18.0  35.0    0.514  19.0  22.0    0.864   6.0   
NaN    41.0  90.0    0.456  15.0  39.0    0.385   9.0  15.0    0.600  12.0   
NaN    34.0  95.0    0.358   9.0  39.0    0.231  17.0  21.0    0

<ipython-input-8-f72e0f37175e>:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('team').apply(add_target)
<ipython-input-8-f72e0f37175e>:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['target'].fillna(2, inplace=Tr

In [9]:
df.groupby(["home"]).apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])

#print(df_rolling.head(60).to_string(index=False))

<ipython-input-9-23ecb1782d91>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby(["home"]).apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])


,0
home,
0.0,0.429779
1.0,0.570221


In [10]:
####################################################################################################
# CALCULATE THE AVERAGE FOR THE PREVIOUS SEASONS WITH THE ROLLING WINDOW OF 7 FOR LEARNING THE MODEL #
####################################################################################################

import pandas as pd
import numpy as np

# Define the current season
current_season = 2025

# Filter out the games from the current season
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

#print(df_rolling.columns)

def find_team_averages(team):
    numeric_columns = team.select_dtypes(include=[np.number])  # Select only numeric columns
    rolling = numeric_columns.rolling(ROLLING_WINDOW_SIZE, min_periods=1).mean()  # Calculate rolling mean
    #rolling[['team', 'season']] = team[['team', 'season']]  # Retain 'team' and 'season' columns in the result
    return rolling


# Apply rolling average
#df_rolling = df.copy()
df_rolling.reset_index(drop=True, inplace=True)
df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

# Renaming columns with _7 suffix for numeric columns only
rolling_cols = {col: f"{col}_7" for col in df_rolling.columns if col not in ['team', 'season']} #, 'season','season_rolling','season_original','target']}


# Rename the columns
df_rolling.rename(columns=rolling_cols, inplace=True)
# Display the renamed columns
#print(df_rolling)
#df_rolling.to_csv("D:\\1. Python\\1. NBA Script\\2025\\Gathering_Data\\Whole_Statistic\\df_rolling.csv", index=False)




<ipython-input-10-0e5c0f5d32b5>:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)


In [11]:
df = df.reset_index(drop=True)
df_rolling = df_rolling.reset_index(drop=True)

df = pd.concat([df, df_rolling], axis=1)

#df.to_csv("D:\\1. Python\\1. NBA Script\\2025\\Gathering_Data\\Whole_Statistic\\df_pd.concat.csv", index=False)


df = df.dropna()

print(df)

target_2_rows = df[df['target'] == 2]['target']
print(target_2_rows)

             fg       fga       fg%        3p       3pa       3p%        ft  \
0      0.391304  0.323529  0.656339  0.275862  0.348485  0.351544  0.272727   
1      0.500000  0.338235  0.736429  0.344828  0.303030  0.495249  0.409091   
2      0.369565  0.338235  0.631584  0.275862  0.287879  0.413302  0.386364   
3      0.391304  0.411765  0.000208  0.241379  0.378788  0.286223  0.295455   
4      0.391304  0.441176  0.598091  0.241379  0.257576  0.395487  0.386364   
...         ...       ...       ...       ...       ...       ...       ...   
22709  0.434783  0.441176  0.000227  0.241379  0.363636  0.000297  0.250000   
22710  0.369565  0.279412  0.000261  0.379310  0.469697  0.000373  0.454545   
22711  0.434783  0.455882  0.000221  0.344828  0.500000  0.000321  0.454545   
22712  0.347826  0.367647  0.000197  0.275862  0.439394  0.000287  0.409091   
22713  0.478261  0.397059  0.000283  0.310345  0.424242  0.000334  0.522727   

            fta       orb       drb  ...  ast%_max_

In [12]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    # Ensure the 'team' column is not part of the index and is correctly formatted
    if 'team' in df.columns:
        return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))
    else:
        raise KeyError("The 'team' column is missing or not properly formatted in the DataFrame.")

# Ensure the 'team' column exists and is not part of the index
if 'team' not in df.columns:
    print("The 'team' column is missing. Ensure the column is present in your DataFrame.")

# Reset the index to avoid potential issues with multi-indexing
df = df.reset_index(drop=True)

# Add shifted columns for "home", "team_opp", and "date"
df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")

# Drop rows where any of the next columns contain NaN values (optional)
#df = df.dropna(subset=["home_next", "team_opp_next", "date_next"])

# Optionally, save the DataFrame to a CSV file
#df.to_csv("D:\\1. Python\\1. NBA Script\\2025\\Gathering_Data\\Whole_Statistic\\df_dropna_target_2.csv", index=False)
#df.to_csv("D:\\1. Python\\1. NBA Script\\2025\\Gathering_Data\\Whole_Statistic\\df.csv", index=False)


# Display the first few rows to check the output
#print(df.head())

target_2_rows = df[df['target'] == 2]['target']
print(target_2_rows)


752      2
1598     2
2332     2
3051     2
3757     2
4537     2
5305     2
6092     2
6803     2
7624     2
8390     2
9144     2
9917     2
10668    2
11414    2
12211    2
13002    2
13739    2
14468    2
15196    2
15957    2
16681    2
17454    2
18212    2
18968    2
19684    2
20431    2
21221    2
21979    2
22713    2
Name: target, dtype: int64


<ipython-input-12-1e3731c6949c>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))
<ipython-input-12-1e3731c6949c>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))
<ipython-input-12-1e3731c6949c>:8: DeprecationWarning: DataFrameGroupBy.apply op

In [13]:
for _, game in games_df.iterrows():
    home_team = game['home_team']
    away_team = game['away_team']
    game_day = game['game_date']

    print(home_team)
    print(away_team)
    print(game_day)


    last_home_team_index = df.loc[df['team'] == home_team].iloc[::-1].index[0]

    df.loc[last_home_team_index, 'team_opp_next'] = away_team
    df.loc[last_home_team_index, 'home_next'] = 1
    df.loc[last_home_team_index, 'date_next'] = game_day


    last_away_team_index = df.loc[df['team'] == away_team].iloc[::-1].index[0]

    df.loc[last_away_team_index, 'team_opp_next'] = home_team
    df.loc[last_away_team_index, 'home_next'] = 0
    df.loc[last_away_team_index, 'date_next'] = game_day


BRK
ORL
2024-12-01
MEM
IND
2024-12-01
CLE
BOS
2024-12-01
NYK
NOP
2024-12-01
TOR
MIA
2024-12-01
HOU
OKC
2024-12-01
UTA
LAL
2024-12-01
POR
DAL
2024-12-01
SAC
SAS
2024-12-01
LAC
DEN
2024-12-01


In [14]:
# Merging DataFrames
# Convert rolling_cols dictionary keys to a list and add other columns
full = df.merge(df[list(rolling_cols.keys()) + ["team_opp_next", "date_next", "team"]],
                left_on=["team", "date_next"],
                right_on=["team_opp_next", "date_next"])


# Save the merged DataFrame
#output_path = "D:\\1. Python\\1. NBA Script\\2025\\Gathering_Data\\Whole_Statistic\\full_new.csv"
#full.to_csv(output_path, index=False)
#print(f"Merged data saved to: {output_path}")

# Display basic info and first few rows of the merged DataFrame
print("Full DataFrame Info:")
print(full.info())
print("\nFirst few rows of the merged DataFrame:")
print(full.head())

# Print number of rows in the merged DataFrame
num_rows = full.shape[0]
print(f"Number of rows in 'full' DataFrame: {num_rows}")

# Extract and print rows with target == 2
target_2_rows = full[full['target'] == 2]['target']
print("\nRows where 'target' == 2:")
print(target_2_rows)


Full DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22876 entries, 0 to 22875
Columns: 402 entries, fg_x to team_y
dtypes: bool(1), float64(392), int64(2), object(7)
memory usage: 70.0+ MB
None

First few rows of the merged DataFrame:
       fg_x     fga_x     fg%_x      3p_x     3pa_x     3p%_x      ft_x  \
0  0.391304  0.323529  0.656339  0.275862  0.348485  0.351544  0.272727   
1  0.500000  0.338235  0.736429  0.344828  0.303030  0.495249  0.409091   
2  0.369565  0.338235  0.631584  0.275862  0.287879  0.413302  0.386364   
3  0.391304  0.411765  0.000208  0.241379  0.378788  0.286223  0.295455   
4  0.391304  0.441176  0.598091  0.241379  0.257576  0.395487  0.386364   

      fta_x     orb_x     drb_x  ...  stl%_max_opp_y  blk%_max_opp_y  \
0  0.234375  0.241379  0.386364  ...           0.034           0.068   
1  0.406250  0.241379  0.363636  ...           0.061           0.079   
2  0.343750  0.275862  0.477273  ...           0.047           0.045   
3  0.21

In [15]:
mask = full['date_next'] == game_day
#filtered_df = full.loc[mask, ['team_x', 'team_opp_next_x', 'team_y', 'team_opp_next_y', 'date_next', 'home_next']]
filtered_df = full.loc[mask, ['team_x', 'team_opp_next_x', 'team_y', 'team_opp_next_y', 'date_next', 'home_next']]

print(filtered_df)

      team_x team_opp_next_x team_y team_opp_next_y   date_next  home_next
1609     BOS             CLE    CLE             BOS  2024-12-01        0.0
2351     BRK             ORL    ORL             BRK  2024-12-01        1.0
4568     CLE             BOS    BOS             CLE  2024-12-01        1.0
5344     DAL             POR    POR             DAL  2024-12-01        0.0
6139     DEN             LAC    LAC             DEN  2024-12-01        0.0
8456     HOU             OKC    OKC             HOU  2024-12-01        1.0
9218     IND             MEM    MEM             IND  2024-12-01        0.0
9995     LAC             DEN    DEN             LAC  2024-12-01        1.0
10750    LAL             UTA    UTA             LAL  2024-12-01        0.0
11500    MEM             IND    IND             MEM  2024-12-01        1.0
12305    MIA             TOR    TOR             MIA  2024-12-01        0.0
14573    NOP             NYK    NYK             NOP  2024-12-01        0.0
15303    NYK             

In [16]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns


In [17]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]
selected_features = selected_columns.unique()

selected_features

Index(['fg_x', 'fga_x', 'fg%_x', '3p_x', '3pa_x', '3p%_x', 'ft_x', 'fta_x',
       'orb_x', 'drb_x',
       ...
       'trb%_max_opp_y', 'ast%_max_opp_y', 'stl%_max_opp_y', 'blk%_max_opp_y',
       'tov%_max_opp_y', 'usg%_max_opp_y', 'ortg_max_opp_y', 'drtg_max_opp_y',
       'total_opp_y', 'home_opp_y'],
      dtype='object', length=391)

In [18]:
full_train = full[full["target"] != 2]
full_pred = full[full["target"] == 2]

print(full_pred)

X = full_train[selected_features].values
y = full_train["target"].values


           fg_x     fga_x     fg%_x      3p_x     3pa_x     3p%_x      ft_x  \
1609   0.652174  0.558824  0.000325  0.793103  0.757576  0.000506  0.386364   
2351   0.413043  0.250000  0.000316  0.482759  0.378788  0.000574  0.227273   
4568   0.413043  0.485294  0.000192  0.379310  0.515152  0.000343  0.318182   
5344   0.478261  0.441176  0.000261  0.517241  0.530303  0.000457  0.204545   
6139   0.586957  0.352941  0.000395  0.379310  0.439394  0.000395  0.431818   
8456   0.456522  0.411765  0.000259  0.344828  0.409091  0.000384  0.727273   
9218   0.478261  0.367647  0.000299  0.413793  0.439394  0.000432  0.272727   
9995   0.369565  0.352941  0.000221  0.344828  0.409091  0.000384  0.227273   
10750  0.304348  0.191176  0.000255  0.379310  0.515152  0.000343  0.363636   
11500  0.608696  0.529412  0.000310  0.413793  0.560606  0.000348  0.318182   
12305  0.521739  0.500000  0.000262  0.724138  0.757576  0.000462  0.318182   
14573  0.413043  0.455882  0.000205  0.310345  0.378

In [19]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'num_leaves': [10, 20, 30],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 5, 10]
}

# Create a LightGBM classifier
base_estimator = lgb.LGBMClassifier(objective='binary',
                                     metric='auc',
                                     boosting_type='gbdt',
                                     verbosity=-1,
                                     random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=base_estimator,
                           param_grid=param_grid,
                           scoring='roc_auc',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

# Perform grid search
#grid_search.fit(X_train, y_train)

# Print the best parameters
#print("Best parameters found:", grid_search.best_params_)


In [21]:
#Best parameters found: {'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 5, 'num_leaves': 10}

params = {
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 10,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'bagging_freq': 10,
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'random_state': 42,
    'lambda_l1': 0.5,
    'lambda_l2': 0.5,
    'max_depth': 7,
    'min_child_weight': 5
}

model = lgb.LGBMClassifier(**params)

In [22]:
# Train the model using X_train and y_train
model.fit(X_train, y_train)

# Predict the target values for the test set X_test
y_pred = model.predict(X_test)

# Check the accuracy of the model using the test set
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 60.98%


In [23]:
importances = model.feature_importances_

# create a dictionary to store feature importances with column names
feat_importances = dict(zip(selected_columns, importances))

# sort the dictionary by importance score in descending order
sorted_feat_importances = sorted(feat_importances.items(), key=lambda x: x[1], reverse=True)


# Print the sorted feature importances
for feature, importance in sorted_feat_importances[:30]:
    print("{}: {}".format(feature, importance))


home_next: 23
ortg_7: 20
pts_max_y: 19
pts_max_7: 18
drtg_7: 16
usg%_max_y: 16
usg%_max_7: 11
ortg_max_7: 10
fg_opp_7: 10
blk_opp_7: 10
drtg_max_opp_7: 10
3pa_7: 9
ast_7: 9
pf_y: 9
orb%_max_7: 8
ft_opp_7: 8
ast_opp_7: 8
3pa_max_opp_7: 8
pts_opp_y: 8
ast%_7: 7
stl%_7: 7
fta_max_7: 7
trb_max_7: 7
drtg_max_7: 7
pts_opp_7: 7
ortg_y: 7
drtg_y: 7
blk_opp_y: 7
3par_x: 6
3p%_opp_x: 6


In [24]:
# predict on new data
full_pred["proba"] = model.predict_proba(full_pred[selected_features])[:,1]
full_pred["proba"]

<ipython-input-24-22cd410e1e14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_pred["proba"] = model.predict_proba(full_pred[selected_features])[:,1]


,proba
1609,0.600573
2351,0.418875
4568,0.562646
5344,0.420172
6139,0.418192
8456,0.677178
9218,0.417980
9995,0.670165
10750,0.445253
11500,0.628289


In [25]:
home_teams_prob = list(games_df['home_team'])
away_teams_prob = list(games_df['away_team'])

#print(home_teams_prob)
#print(away_teams_prob)

# Filter the rows where team_x is a home team
full_pred_prob = full_pred['team_x'].isin(home_teams_prob)
#print(full_pred_prob)

#full_pred_prob = full_pred['team_x'].isin(home_teams_prob)
full_pred[full_pred_prob]['proba']


,proba
2351,0.418875
4568,0.562646
8456,0.677178
9995,0.670165
11500,0.628289
15303,0.778141
19097,0.362633
19823,0.592229
21376,0.645354
22138,0.371903


In [26]:
# Filter rows where full_pred_prob is True

team_x = full_pred.loc[full_pred_prob, 'team_x']
team_y = full_pred.loc[full_pred_prob, 'team_y']
#print(team_x)
#print(team_y)

team_pairs = pd.concat([team_x, team_y], axis=1)


print(team_pairs)

      team_x team_y
2351     BRK    ORL
4568     CLE    BOS
8456     HOU    OKC
9995     LAC    DEN
11500    MEM    IND
15303    NYK    NOP
19097    POR    DAL
19823    SAC    SAS
21376    TOR    MIA
22138    UTA    LAL


In [27]:
# Filter rows where full_pred_prob is True
home_team_preds = full_pred.loc[full_pred_prob, ['team_x', 'team_y', 'proba']]

# Rename the columns to match the format of the games_df dataframe
home_team_preds.columns = ['home_team', 'away_team', 'home_team_prob']#, 'odds', 'result']
home_team_preds['odds 1'] = 0
home_team_preds['odds 2'] = 0

home_team_preds['result'] = 0
#home_team_preds['hit'] = 0
home_team_preds['date'] = game_day

# Print the resulting dataframe
print((home_team_preds).to_string(index=False))

#print(combined_df.head(80).to_string(index=False))



home_team away_team  home_team_prob  odds 1  odds 2  result       date
      BRK       ORL        0.418875       0       0       0 2024-12-01
      CLE       BOS        0.562646       0       0       0 2024-12-01
      HOU       OKC        0.677178       0       0       0 2024-12-01
      LAC       DEN        0.670165       0       0       0 2024-12-01
      MEM       IND        0.628289       0       0       0 2024-12-01
      NYK       NOP        0.778141       0       0       0 2024-12-01
      POR       DAL        0.362633       0       0       0 2024-12-01
      SAC       SAS        0.592229       0       0       0 2024-12-01
      TOR       MIA        0.645354       0       0       0 2024-12-01
      UTA       LAL        0.371903       0       0       0 2024-12-01


In [37]:
team_preds = "nba_games_predict_" + TODAY + ".csv"
print(team_preds)

directory_path = os.path.join(season_dir, f"LightGBM")

os.makedirs(directory_path, exist_ok=True)

# Construct the full file path
full_path = os.path.join(directory_path, team_preds)

# ... (rest of your code remains the same) ...

# Save the DataFrame to the CSV file in the specified directory
home_team_preds.to_csv(full_path, index=False)
print(f"Predictions saved to: {full_path}")

nba_games_predict_2024-12-01.csv
Predictions saved to: /content/drive/My Drive/NBA_Script/2025/LightGBM/nba_games_predict_2024-12-01.csv
